In [ ]:
import os
import json
import sqlite3
import pandas as pd
from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [ ]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [ ]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [ ]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [ ]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [ ]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [ ]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [ ]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

### Migrate ObstetricsHistory (INSERT)

In [ ]:
source_tablename = 'tblobstetrichistory'
target_tablename = 'history_obstetrics'
data_index = 11 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### tblobstetrichistory_x NOT TO BE MIGRATED

### Migrate Short Pregnancies History data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory'
target_tablename = 'pregnancy_history'
data_index = 16 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### Migrate Short Pregnancies History X data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory_x'
target_tablename = 'pregnancy_history_x'
data_index = 17 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)
# delete_records(target_tablename, db_path)

Delete records with null TER

In [ ]:


conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
DELETE FROM pregnancy_history_x
WHERE pregnancy_id IN (SELECT id FROM pregnancy_history WHERE ter IS NULL)
""")
cursor.execute("""
DELETE FROM pregnancy_history
WHERE ter IS NULL
""")
conn.commit()

conn.close()

### Migrate Gyn History X data (INSERT)

DELETE 5,8 and 10 records

In [ ]:
source_tablename = 'tblgynhistory'
target_tablename = 'gyn_history'
data_index = 6 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("""
    UPDATE gyn_history
    SET comments = TRIM(REPLACE(REPLACE(comments, '\n\n', '\n'), '\r\n\r\n', '\n'))
    WHERE comments IS NOT NULL
""")
conn.commit()
conn.close()

## Update next_suggested_appointment date

In [ ]:
from fetusapp.chatai.note_to_app_service import analyze_doctor_note
from loguru import logger

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
               SELECT patient_id
                    , date_of_visit
                    , next_suggested_appointment
                    , comments
               FROM gyn_history g
               JOIN patients p ON g.patient_id = p.id
               WHERE date_of_visit >= date('2024-06-13', '-1 year')
               AND g.is_active = 1
               AND p.is_active = 1
               AND g.created_by = 1
               GROUP BY patient_id
               ORDER BY 2 DESC;
               """)
visits = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

visits_df = pd.DataFrame(visits, columns=columns)
logger.info(f"Total visits to analyze: {len(visits_df)}")

In [ ]:
visits_df

In [ ]:
from fetusapp import app  # Make sure this imports your Flask app

with app.app_context():
    for i, row in visits_df.head(50).iterrows():
        logger.info("="*80)
        logger.info(f"Analyzing comment #{i}:\n{row['comments']}...")
        doctor_note = row['comments']
        patient_id = row['patient_id']
        current_appointment_date = row['date_of_visit']
        logger.info(f"Using current appointment date: {current_appointment_date}")
        try:
            analyze_doctor_note(doctor_note, patient_id, current_appointment_date=current_appointment_date)
        except Exception as e:
            logger.error(f"Error analyzing note for patient {patient_id}: {e}")
        logger.info("="*80)

### Migrate Medical Exams (INSERT)

In [ ]:
source_tablename = 'tblmedexam'
target_tablename = 'medical_exams'
data_index = 8 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
# delete_records(target_tablename, db_path)
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)

### ... to be continued

In [ ]:
get_tables(db_path)

--------------